In [1]:
from keras.datasets import imdb
top_words= 10000
((x_train, y_train),(x_test,y_test))=imdb.load_data(num_words=top_words, seed= 21)

Using TensorFlow backend.


In [2]:
x_train.shape

(25000,)

In [3]:
x_test.shape

(25000,)

In [4]:
import numpy as np
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([12500, 12500], dtype=int64))

In [5]:
word_to_id = {w: i+3 for w,i in imdb.get_word_index().items()}
id_to_word = {0:'<PAD>',1:'<START>',2:'<UNK>'}
id_to_word.update({i+3: w for w,i in imdb.get_word_index().items()})

def convert_to_text(sequence):
    return ' '.join([id_to_word[s] for s in sequence if s>=3])

print(convert_to_text(x_train[8]))

this movie was like a bad train wreck as horrible as it was you still had to continue to watch my boyfriend and i rented it and wasted two hours of our day now don't get me wrong the acting is good just the movie as a whole just both of us there wasn't anything positive or good about this scenario after this movie i had to go rent something else that was a little lighter jennifer is as usual a very dramatic actress her character seems manic and not all there hannah though over played she does a wonderful job playing out the situation she is in more than once i found myself yelling at the tv telling her to fight back or to get violent all in all very violent movie not for the faint of heart


In [6]:
from keras.preprocessing.sequence import pad_sequences
max_pad = 200
x_train = pad_sequences(x_train, maxlen=max_pad)
x_test = pad_sequences(x_test, maxlen=max_pad)
print(x_train[30])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    1   12  421   40   13  296   14   20 3207  211  159
   12   16  427  727  175   58    4   65  805    8   79    6  227 2509
  175   58   13    2  142  221    8  593   13  219  164   21   51   13
  873   40    4 5375 3283 3057   56  160 8199   88   50   16   24  195
 5375   11  107  715 1035   65   15   47  413    6 1591   78  116   78
  318  302   64    4  995 1766 1002    2   16    6  227  163  137    2
  187    4   78  493    4 2446    7    2   16   38  966  379   15   12
   93   72  462   33   90   18    6  561   13  594  138   13  437   61
   58 9162   23   15  902    7    6   20   12  528   79  433   88   12
    9 

In [7]:
from keras.models import Sequential
from keras.layers import Bidirectional,Dense,Dropout,GlobalMaxPool1D,LSTM
from keras.layers.embeddings import Embedding

embedding_vector_length =32

model =Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_pad))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 32)           320000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 128)          49664     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 371,745
Trainable params: 371,745
Non-trainable params: 0
_____

In [8]:
history = model.fit(x_train, y_train,validation_data = (x_test, y_test),epochs = 3,batch_size=256)


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 154s 6ms/step - loss: 0.5826 - accuracy: 0.6669 - val_loss: 0.4237 - val_accuracy: 0.8257
Epoch 2/3
25000/25000 [==============================] - 154s 6ms/step - loss: 0.3300 - accuracy: 0.8670 - val_loss: 0.3486 - val_accuracy: 0.8481
Epoch 3/3
25000/25000 [==============================] - 178s 7ms/step - loss: 0.2261 - accuracy: 0.9139 - val_loss: 0.3630 - val_accuracy: 0.8558


In [9]:
loss, metric = model.evaluate(x_test,y_test, verbose=0)
print("Test accuracy: %0.3f"% metric)

Test accuracy: 0.856


In [10]:
y_train.shape

(25000,)

In [11]:
x_test.shape

(25000, 200)

In [12]:
y_test.shape

(25000,)